In [15]:
import numpy as np
import pandas as pd

In [16]:
"""
Looking at population density and distribution of people

Dataset:
Base Zoning: http://opendata.columbus.gov/datasets/96f7642a62f84db997f9e1db4a776995_4
    - Can look at zoning to see which locations are more populated
    
BZA Zoning Variances: http://opendata.columbus.gov/datasets/19786dd084e644a4aea6b33f867dd631_1
    - More Zoning?
    
Recommended Land Use: http://opendata.columbus.gov/datasets/26f0606f94db4c07a63aef3cc8927c9b_21
    - Where we can build charging stations?
    
Population Density Map: https://apps.morpc.org/census2010/

""";

In [28]:
%ls

Census Blockgroup 2010.csv  PT_data.ipynb
Census_Blockgroup_2010.csv  Points_of_Interest.csv
NS_data.ipynb               README.md


In [34]:
population_data = pd.read_csv("Columbus_2010_Population.csv")

In [35]:
print(len(population_data))
population_data.head()

1088


,GEOID,2010 Total Population,2010 White Population,2010 African American Population,2010 Asian Population,2010 Other Races Population,2010 Hispanic Population,2010 Total Households,2010 Occupied Households,2010 Vacant Households,2000 Total Population,2000 White Population,2000 African American Population,2000 Asian Population,2000 Other Races Population,2000 Total Housing Units,2000 Occupied Housing Units,2000 Vacant Housing Units,2000 Hispanic Population
0,3.904100e+11,1277,1209,25,5,38,18,576,539,37,1381,1316,30,8,27,573,547,26,9
1,3.904100e+11,1721,1613,50,19,39,32,822,724,98,1716,1638,17,16,45,799,735,64,35
2,3.904100e+11,2258,1896,156,129,77,63,449,397,52,2579,2305,157,66,51,447,414,33,42
3,3.904100e+11,859,819,19,1,20,16,414,373,41,800,777,5,2,16,340,320,20,2
4,3.904100e+11,1002,953,18,3,28,24,454,423,31,1085,1038,13,1,33,443,425,18,13


In [ ]:
# TODO: find a way to associate GEOID with latitude/longitude, somehow associate with city/county?